# Cytoscape Node `FocusView`
Individual widget for viewing the LPG graph for a single RDF node (and its outgoing connections).

Let's start by getting our graph loaded in.

In [ ]:
from ipyradiant import FileManager, PathLoader

lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]

Create the `FocusView` widget and set the initial `subject_uri`.

In [ ]:
from rdflib import URIRef
from ipyradiant.visualization.cytoscape.focus import FocusView


fv = FocusView(
    subject_uri=URIRef("https://swapi.co/resource/human/1"),
    rdf_graph=lw.graph
)
fv

In [ ]:
# Change the subject
fv.subject_uri = URIRef("https://swapi.co/resource/human/6")

Link to JSON widget to inspect node data. This shows that target nodes also have full data collapsed onto node. 

In [ ]:
from IPython.display import JSON, display
import ipywidgets as W


class CombinedExample(W.VBox):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.json_output = W.Output()
        self.interactive_widget = kwargs["interactive_widget"]
        self.children = [self.interactive_widget, self.json_output]
        self.interactive_widget.cytoscape_widget.on("node", "click", self.load_json)

    def load_json(self, node):
        data = node["data"]
        data.pop("_label", None)
        data.pop("_attrs", None)
        with self.json_output:
            self.json_output.clear_output()
            display(JSON(data))

In [ ]:
ce = CombinedExample(interactive_widget=fv.cytoscape_viewer)
ce